In [3]:
!pip install virtualenv
!virtualenv venv
!source venv/bin/activate
!pip install fastapi
!pip install uvicorn
!pip install pydantic
!pip install torch whisper nest-asyncio pyngrok
!pip install transformers
# !pip install git+https://github.com/huggingface/parler-tts.git
!apt get install ffmpeg
!pip install python-multipart
!pip install ipython

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


created virtual environment CPython3.10.13.final.0-64 in 669ms
  creator CPython3Posix(dest=/kaggle/working/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.0.1, setuptools==67.4.0, wheel==0.38.4
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 680.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=83c8e5f8150dd41e46e6095290ea65cccb2e68eb5190e50731729d682d600795
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
E: Invalid operation get


In [4]:
!ngrok config add-authtoken # your ngrok access token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [5]:
import torch
torch.cuda.empty_cache()

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from transformers import pipeline
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import io
import base64

app = FastAPI()

# Initialize the list of allowed origins with localhost
origins = ["http://localhost:3000"]

# Dynamically get the public URL from ngrok and add it to allowed origins
public_url = ngrok.connect(8000)
print("Public URL:", public_url)
origins.append(public_url)

# Apply the CORS settings with the updated origins list
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define your models and routes
stt_model = pipeline("automatic-speech-recognition", model="openai/whisper-tiny.en", device = 0)
llm_model = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device = 0,
    max_new_tokens=50,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)
tts_model = pipeline("text-to-speech", model="suno/bark-small", device = 0)

@app.post("/api/transcribe")
async def transcribe_audio(file: UploadFile = File(...)):
    audio_bytes = await file.read()
    transcription = stt_model(audio_bytes)["text"]
    llm_result = llm_model(transcription)[0]['generated_text'].split('\n\n')[1]
    tts_result = tts_model(llm_result)
    audio_base64 = base64.b64encode(tts_result["audio"]).decode('utf-8')
    return {"speaked_text": transcription, "generated_text": llm_result, "audio_np": audio_base64}

# Run the FastAPI app
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: NgrokTunnel: "https://2ee8-35-184-209-81.ngrok-free.app" -> "http://localhost:8000"


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

INFO:     Started server process [34]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
import IPython.display as ipd
import numpy as np
tts_model = pipeline("text-to-speech", model="suno/bark-small", device = 0)
tts_result = tts_model('hi, Hello')
# If the output is in the form of an audio array
audio_array = np.array(tts_result['audio'], dtype=np.float32)

# Play the audio in Colab
ipd.display(ipd.Audio(audio_array, rate=24000))
